### Library

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import pandas as pd
import time
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import ResNet50

### Some constant

Note: macOS cannot quit properly in jupyter notebook

#### Change the cv_flag to switch between live camera and video

In [2]:
# cv_flag = "test.mp4"
cv_flag = 0

In [3]:
deploy_path = "./res_ssd/deploy.prototxt"
model_path = "./res_ssd/res10_300x300_ssd_iter_140000.caffemodel"
deep_face_database = "./database/"


### Remove previous represenation in database folder

In [4]:
if os.path.exists(deep_face_database + "representations_fbdeepface_model.pkl"):
    os.remove(deep_face_database + "representations_fbdeepface_model.pkl")

### Function to detect the bounding box of a face

In [5]:
OFFSET = 20
def detect(frame, faceNet):
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 
                                 1.0, 
                                 (224, 224),
                                 (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()

    locs = []
    faces = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX-OFFSET), max(0, startY-OFFSET))
            (endX, endY) = (min(w - 1, endX+OFFSET), min(h - 1, endY+OFFSET))

            face = frame[startY:endY, startX:endX]
            face = cv2.resize(face, (224, 224))

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    return locs, faces

### Function  to load deepface module

In [6]:
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
vgg_model = VGGFace.loadModel()
def get_deepFace_result(face):
    cv2.imwrite('tmp.jpg', face)
    df = DeepFace.find(img_path="tmp.jpg", 
                       db_path=deep_face_database, 
                       enforce_detection=False, 
                       model_name = 'VGG-Model',
                       model = vgg_model,
                       distance_metric = 'cosine')
#     df = DeepFace.find(img_path="tmp.jpg", 
#                        db_path=deep_face_database, 
#                        enforce_detection=False)
    id = "unknown"
    if df.shape[0] >0:
        id = df.iloc[0].identity
        print(id)
        print(os.path.split(id)[0])
        id = os.path.split(id)[0].replace(deep_face_database,'')
    return id

### Load mask detection model

In [7]:
from tensorflow.keras.models import load_model
mask_model = load_model('face_mask.h5')

### Use OpenCV to open video stream and do the detection

In [ ]:
detector = cv2.dnn.readNetFromCaffe(deploy_path, model_path)
source=cv2.VideoCapture(cv_flag)
while(source.isOpened()):
    ret,image=source.read()
    if ret == False:
        break
    locs, faces = detect(image, detector)
    for (box, face) in zip(locs, faces):
        (startX, startY, endX, endY) = box
        resize_img  = cv2.resize(face,(150,150))
        normalized = resize_img/255.0
        reshape = np.reshape(normalized,(1,150,150,3))
        reshape = np.vstack([reshape])
        result = mask_model.predict(reshape)
        prediction = np.argmax(result,axis=1)
        color = (0, 255, 0)
        label = "Mask"
        if prediction[0] == 1:
            color = (0, 0, 255)
            label = "No Mask"
       
        
        path = get_deepFace_result(face)
        cv2.putText(image, path, (startX, startY - 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.putText(image, label, (startX, startY - 10),
                cv2.FONT_HERSHEY_COMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        cv2.imshow('frame',image)
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()
source.release()

Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

Already built model is passed
There are  5  representations found in  representations_vgg_model.pkl


Analyzing: 100%|██████████| 1/1 [00:00<00:00,  1.78it/s]


find function lasts  0.574012041091919  seconds
./database/Zirong Yu/front_nomask.jpg
./database/Zirong Yu


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

...

find function lasts  0.22799181938171387  seconds
./database/Zirong Yu/front_mask2.jpg
./database/Zirong Yu
Already built model is passed
There are  5  representations found in  representations_vgg_model.pkl


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

find function lasts  0.210968017578125  seconds
./database/Zirong Yu/front_mask2.jpg
./database/Zirong Yu
Already built model is passed
There are  5  representations found in  representations_vgg_model.pkl


Analyzing: 100%|██████████| 1/1 [00:00<00:00,  4.12it/s]


find function lasts  0.24580001831054688  seconds
./database/Zirong Yu/1606259394031.jpg
./database/Zirong Yu
